# Homework 2

In [3]:
#Import packages

# general use imports
%pylab inline
import datetime
import numpy as np
import os
import six
import warnings
import matplotlib.pyplot as plt
import re

# pandas-related imports
from __future__ import print_function
import pandas as pd
import scipy
import sklearn

# configure pandas display: set the maximum number of columns displayed to 25
pd.options.display.max_columns = 25

# record linkage package
import recordlinkage as rl
from recordlinkage.preprocessing import clean, phonenumbers, phonetic

# CSV file reading-related imports
import csv

# create postgresql connection - three '/' indicate to use default host, port, username, and password
from sqlalchemy import create_engine
engine = create_engine('postgresql:///ada_pub')

print( "Imports loaded at " + str( datetime.datetime.now() ) )

Populating the interactive namespace from numpy and matplotlib
Imports loaded at 2018-02-23 14:49:13.561029


## HOMEWORK
## Clean Projects - original file
Load dataframe from database. Then clean the names. clean names, zipcode, and a year variable.

Now we will clean and preprocess the grants Data. This will be your assignment. We need clean names, zipcode, and a year variable. You can start working on this when we are done with the linkage.

In [4]:
# SELECT contact_pi_project_leader, fy, organization_country, organization_zip, project_title, project_id
# FROM projects

# Define the query
QUERY = '''
SELECT contact_pi_project_leader, fy, organization_country, organization_zip, project_title, project_id
FROM projects;
'''
# Save the output in a dataframe
projects=pd.read_sql_query(QUERY,con=engine)

projects.head(100)

,contact_pi_project_leader,fy,organization_country,organization_zip,project_title,project_id
0,"TREBAK, MOHAMED",2013,UNITED STATES,122220001,RECEPTOR-REGULATED CA2+ ENTRY PATHWAYS IN SMOO...,484322
1,"ABRAHAM, EDATHARA C",2013,UNITED STATES,722057101,MODIFICATION OF ALPHA-CRYSTALLIN CHAPERONE FUN...,484323
2,"GUZMAN, RAUL J",2013,UNITED STATES,372407749,ROLE OF ARTERIAL CALCIFICATION IN RESTENOSIS,484324
3,"JAVIER, RONALD T.",2013,UNITED STATES,770303411,MECHANISMS OF THE ADENOVIRUS E4-ORF1 ONCOPROTEIN,484325
4,"HUMPHREY, JAY D.",2013,UNITED STATES,065208327,"MULTISCALE, MULTIPHYSICS MODEL OF THROMBUS BIO...",484326
5,"DEGROAT, WILLIAM C",2013,UNITED STATES,152132303,SPINAL MECHANISMS CONTROLLING EXTERNAL URETHRA...,484332
6,"DEADWYLER, SAMUEL A.",2013,UNITED STATES,271570001,NEUROIMAGING CORRELATES OF COCAINE REINFORCEME...,484327
7,"STEYGER, PETER STEPHEN",2013,UNITED STATES,972393098,COCHLEAR TRAFFICKING AND HAIR CELL UPTAKE OF A...,484328
8,"WEERSING, V ROBIN",2013,UNITED STATES,921821901,1/2-BRIEF CBT FOR PEDIATRIC ANXIETY AND DEPRES...,484329
9,"DEPINHO, RONALD ANTHONY",2013,UNITED STATES,770304009,GENETICALLY ENGINEERED MOUSE FACILITY,484330


In [6]:
# Remove Whitespaces
#Lambda creates a new variable, replaces the olde one without blank spaces
projects.rename(columns=lambda x: x.strip(), inplace=True)
projects.head()

,contact_pi_project_leader,fy,organization_country,organization_zip,project_title,project_id
0,"TREBAK, MOHAMED",2013,UNITED STATES,122220001,RECEPTOR-REGULATED CA2+ ENTRY PATHWAYS IN SMOO...,484322
1,"ABRAHAM, EDATHARA C",2013,UNITED STATES,722057101,MODIFICATION OF ALPHA-CRYSTALLIN CHAPERONE FUN...,484323
2,"GUZMAN, RAUL J",2013,UNITED STATES,372407749,ROLE OF ARTERIAL CALCIFICATION IN RESTENOSIS,484324
3,"JAVIER, RONALD T.",2013,UNITED STATES,770303411,MECHANISMS OF THE ADENOVIRUS E4-ORF1 ONCOPROTEIN,484325
4,"HUMPHREY, JAY D.",2013,UNITED STATES,065208327,"MULTISCALE, MULTIPHYSICS MODEL OF THROMBUS BIO...",484326


In [12]:
# Getting the first character in the cleaned name variable
projects['name_last'] = projects.contact_pi_project_leader.str.split(', ').str.get(0)
projects['name_first'] = projects.contact_pi_project_leader.str.split(', ').str.get(1)
#Split first name into first and middle
projects['name_middle'] = projects.name_first.str.split(' ').str.get(1)
projects['name_first'] = projects.name_first.str.split(' ').str.get(0)

projects.head()

,contact_pi_project_leader,fy,organization_country,organization_zip,project_title,project_id,name_last,name_first,name_middle
0,"TREBAK, MOHAMED",2013,UNITED STATES,122220001,RECEPTOR-REGULATED CA2+ ENTRY PATHWAYS IN SMOO...,484322,TREBAK,MOHAMED,NaN
1,"ABRAHAM, EDATHARA C",2013,UNITED STATES,722057101,MODIFICATION OF ALPHA-CRYSTALLIN CHAPERONE FUN...,484323,ABRAHAM,EDATHARA,C
2,"GUZMAN, RAUL J",2013,UNITED STATES,372407749,ROLE OF ARTERIAL CALCIFICATION IN RESTENOSIS,484324,GUZMAN,RAUL,J
3,"JAVIER, RONALD T.",2013,UNITED STATES,770303411,MECHANISMS OF THE ADENOVIRUS E4-ORF1 ONCOPROTEIN,484325,JAVIER,RONALD,T.
4,"HUMPHREY, JAY D.",2013,UNITED STATES,065208327,"MULTISCALE, MULTIPHYSICS MODEL OF THROMBUS BIO...",484326,HUMPHREY,JAY,D.


In [14]:
# Cleaning names (using the record linkage package tool, see imports)
# Clean removes any characters such as '-', '.', '/', '\', ':', brackets of all types. 
projects['name_last']=clean(projects['name_last'], lowercase=False, remove_brackets=False)
projects['name_first']=clean(projects['name_first'], lowercase=False, remove_brackets=False)
projects['name_middle']=clean(projects['name_middle'], lowercase=False, remove_brackets=False)
projects.head()

,contact_pi_project_leader,fy,organization_country,organization_zip,project_title,project_id,name_last,name_first,name_middle
0,"TREBAK, MOHAMED",2013,UNITED STATES,122220001,RECEPTOR-REGULATED CA2+ ENTRY PATHWAYS IN SMOO...,484322,TREBAK,MOHAMED,NaN
1,"ABRAHAM, EDATHARA C",2013,UNITED STATES,722057101,MODIFICATION OF ALPHA-CRYSTALLIN CHAPERONE FUN...,484323,ABRAHAM,EDATHARA,C
2,"GUZMAN, RAUL J",2013,UNITED STATES,372407749,ROLE OF ARTERIAL CALCIFICATION IN RESTENOSIS,484324,GUZMAN,RAUL,J
3,"JAVIER, RONALD T.",2013,UNITED STATES,770303411,MECHANISMS OF THE ADENOVIRUS E4-ORF1 ONCOPROTEIN,484325,JAVIER,RONALD,T
4,"HUMPHREY, JAY D.",2013,UNITED STATES,065208327,"MULTISCALE, MULTIPHYSICS MODEL OF THROMBUS BIO...",484326,HUMPHREY,JAY,D


In [15]:
# Upcasing names
projects['name_last']=projects.name_last.str.upper()
projects['name_first']=projects.name_first.str.upper()
projects['name_middle']=projects.name_middle.str.upper()

In [16]:
projects.head(100)

,contact_pi_project_leader,fy,organization_country,organization_zip,project_title,project_id,name_last,name_first,name_middle
0,"TREBAK, MOHAMED",2013,UNITED STATES,122220001,RECEPTOR-REGULATED CA2+ ENTRY PATHWAYS IN SMOO...,484322,TREBAK,MOHAMED,NaN
1,"ABRAHAM, EDATHARA C",2013,UNITED STATES,722057101,MODIFICATION OF ALPHA-CRYSTALLIN CHAPERONE FUN...,484323,ABRAHAM,EDATHARA,C
2,"GUZMAN, RAUL J",2013,UNITED STATES,372407749,ROLE OF ARTERIAL CALCIFICATION IN RESTENOSIS,484324,GUZMAN,RAUL,J
3,"JAVIER, RONALD T.",2013,UNITED STATES,770303411,MECHANISMS OF THE ADENOVIRUS E4-ORF1 ONCOPROTEIN,484325,JAVIER,RONALD,T
4,"HUMPHREY, JAY D.",2013,UNITED STATES,065208327,"MULTISCALE, MULTIPHYSICS MODEL OF THROMBUS BIO...",484326,HUMPHREY,JAY,D
5,"DEGROAT, WILLIAM C",2013,UNITED STATES,152132303,SPINAL MECHANISMS CONTROLLING EXTERNAL URETHRA...,484332,DEGROAT,WILLIAM,C
6,"DEADWYLER, SAMUEL A.",2013,UNITED STATES,271570001,NEUROIMAGING CORRELATES OF COCAINE REINFORCEME...,484327,DEADWYLER,SAMUEL,A
7,"STEYGER, PETER STEPHEN",2013,UNITED STATES,972393098,COCHLEAR TRAFFICKING AND HAIR CELL UPTAKE OF A...,484328,STEYGER,PETER,STEPHEN
8,"WEERSING, V ROBIN",2013,UNITED STATES,921821901,1/2-BRIEF CBT FOR PEDIATRIC ANXIETY AND DEPRES...,484329,WEERSING,V,ROBIN
9,"DEPINHO, RONALD ANTHONY",2013,UNITED STATES,770304009,GENETICALLY ENGINEERED MOUSE FACILITY,484330,DEPINHO,RONALD,ANTHONY


In [23]:
projects_clean = projects[['name_first', 'name_middle', 'name_last', 'organization_zip', 'fy']]
projects_clean.head()

,name_first,name_middle,name_last,organization_zip,fy
0,MOHAMED,NaN,TREBAK,122220001,2013
1,EDATHARA,C,ABRAHAM,722057101,2013
2,RAUL,J,GUZMAN,372407749,2013
3,RONALD,T,JAVIER,770303411,2013
4,JAY,D,HUMPHREY,065208327,2013
